In [ ]:
from flask import Flask, request, json

import vk_api
import random
import datetime as dt
import re
import traceback
import time

vk = vk_api.VkApi(token="vk1.a.ayGlVKVYNgSMTGrAHHHvlxrKz5SmhZSYfnsvlQggZAmxgRccVI9PCXtXlat-SuRjZ7mzxnveZbDG4FM0kLzZbZ_U0_cr0vikTFSrGtRuPIJpa1j0tyNCgAHOFshVik71SMCeLw5Hst9UK-79CIQODz6v9OINoKUTUz8lemCwzNn-a3wD7ym2OMOY13UYdysBtqJ_5tP2Z0AGpU_vkNlGpw")

from vk_api.keyboard import VkKeyboard, VkKeyboardColor

application = Flask(__name__)


# 1 уровень клавы
keyboard = VkKeyboard(one_time = False)
keyboard.add_button("Записаться на воркшоп", payload = {"type":"v1_v1"}, color = VkKeyboardColor.POSITIVE)
keyboard.add_line()
keyboard.add_button("Важная информация", payload = {"type":"v1_v2"}, color = VkKeyboardColor.SECONDARY)
keyboard.add_line()
keyboard.add_button("Задать вопрос", payload = {"type":"v1_v3"}, color = VkKeyboardColor.SECONDARY)
keyboard.add_line()
keyboard.add_button("Отменить запись", payload = {"type":"v1_v4"}, color = VkKeyboardColor.NEGATIVE)
keyboard.get_keyboard()

# 2 уровень клавы для записи на воркшоп
keyboard_vorkshop = VkKeyboard(one_time = False)
keyboard_vorkshop.add_button('Серия 1', payload = {"type":"v2_v1"}, color = VkKeyboardColor.PRIMARY)
keyboard_vorkshop.add_button('Серия 2', payload = {"type":"v2_v2"}, color = VkKeyboardColor.PRIMARY)
keyboard_vorkshop.add_line()
keyboard_vorkshop.add_button('Назад', payload = {"type":"v2_v3"}, color = VkKeyboardColor.NEGATIVE)
keyboard_vorkshop.get_keyboard()

# 2 уровень клавы для инфы
keyboard_infa = VkKeyboard(one_time = False)
keyboard_infa.add_button('Программа форума', payload = {"type":"v3_v1"}, color = VkKeyboardColor.PRIMARY)
keyboard_infa.add_line()
keyboard_infa.add_button('Место и карта', payload = {"type":"v3_v2"}, color = VkKeyboardColor.PRIMARY)
keyboard_infa.add_line()
keyboard_infa.add_button('Онлайн-трекер', payload = {"type":"v3_v3"}, color = VkKeyboardColor.PRIMARY)
keyboard_infa.add_line()
keyboard_infa.add_button('Назад', payload = {"type":"v3_v4"}, color = VkKeyboardColor.NEGATIVE)
keyboard_infa.get_keyboard()

# 3 уровень клавы (запись на серию 1)
keyboard_seria1 = VkKeyboard(one_time = False)
keyboard_seria1.add_button('Воркшоп 1', payload = {"type":"v4_v1"}, color = VkKeyboardColor.PRIMARY)                                                          
keyboard_seria1.add_line()
keyboard_seria1.add_button('Воркшоп 2', payload = {"type":"v4_v2"}, color = VkKeyboardColor.PRIMARY)
keyboard_seria1.add_line()
keyboard_seria1.add_button('Воркшоп 3', payload = {"type":"v4_v3"}, color = VkKeyboardColor.PRIMARY)
keyboard_seria1.add_line()
keyboard_seria1.add_button('Вернуться назад', payload = {"type":"v4_v4"}, color = VkKeyboardColor.NEGATIVE)
keyboard_seria1.get_keyboard()

# 3 уровень клавы (запись на серию 2)
keyboard_seria2 = VkKeyboard(one_time = False)
keyboard_seria2.add_button('Воркшоп 1', payload = {"type":"v5_v1"}, color = VkKeyboardColor.PRIMARY)                                                          
keyboard_seria2.add_line()
keyboard_seria2.add_button('Воркшоп 2', payload = {"type":"v5_v2"}, color = VkKeyboardColor.PRIMARY)
keyboard_seria2.add_line()
keyboard_seria2.add_button('Воркшоп 3', payload = {"type":"v5_v3"}, color = VkKeyboardColor.PRIMARY)
keyboard_seria2.add_line()
keyboard_seria2.add_button('Вернуться назад', payload = {"type":"v5_v4"}, color = VkKeyboardColor.NEGATIVE)
keyboard_seria2.get_keyboard()

data_list = []

def writing_file(data_list):    # перезапись файла, иземение инфы
    with open ('data_finpoint.txt', 'w') as write_file:    # перезаписываем файл
        for i in data_list:
            write_file.write(i+'\n')
    write_file.close()

def create_data_list():    # извлечение содержимого файла в список
    data_file = open('data_finpoint.txt', encoding = 'utf-8')
    data_list = []       
    for i in data_file:
        data_list.append(i.replace('\n', ''))
    data_file.close()
    return data_list

@application.route('/', methods = ["POST"])
def main():
    try:
        data = json.loads(request.data)
        if data["type"] == "confirmation":
            return "33406204"

        elif data["type"] == "message_new":
            object = data["object"]["message"]
            id = object["peer_id"]
            body = object["text"]
            from_id = str(object['from_id'])
            date = object["date"]
            msg_id = object["id"]
            if int(str(dt.datetime.now() + dt.timedelta(hours = 3) - dt.datetime.utcfromtimestamp(date))[-9:-7]) > 10:
                    return "ok"
                    
            keyboard.get_keyboard()
          
            if body.lower() == "!инфа":
                vk.method("messages.send", {"peer_id": id, "message": id, "random_id": random.randint(2, 2147483647)})
            if body.lower() == 'начать':
                vk.method("messages.send", {"peer_id": id, "message": "Выбери действие", "random_id": random.randint(2, 2147483647),  "keyboard": keyboard.get_keyboard()})  

            data_finpoint = open('data_finpoint.txt', encoding = 'utf-8')
          
         # id пользователя
            global user_id
            user_id = vk.method("users.get", {"user_ids": id})[0]['id']   

         # 1 уровень
            if object["payload"] == '{"type":"v1_v1"}': # кнопка записаться на воркшоп
                list_id = []
                for i in data_list:
                    list_id.append(i.split(';')[0])
                if str(user_id) not in list_id: # если пользователь ещё не записан (отсутствует в файле)
                        vk.method("messages.send", {"peer_id": id, "message": 'Выбери серию, на которую хочешь записаться', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard_vorkshop.get_keyboard()})  

                elif str(user_id) in list_id:
                    for j in data_list:
                        if str(user_id) == j.split(';')[0]:   # если уже записывался 
                          vshp = j.split(';')[-1]
                          message ='Ты уже записан на ' + vshp
                          vk.method("messages.send", {"peer_id": id, "message": message, "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})
                          break
          
            elif object["payload"] == '{"type":"v1_v2"}':    # важная инфа
                vk.method("messages.send", {"peer_id": id, "message": 'Что именно ты хочешь узнать?', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard_infa.get_keyboard()})  
          
            elif object["payload"] == '{"type":"v1_v3"}': # задать вопрос
                vk.method("messages.send", {"peer_id": id, "message": "Напиши свой вопрос", "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})
           
            elif object["payload"] == '{"type":"v1_v4"}':  # ! отмена записи !              
                list_id = []
                for i in data_list:
                    list_id.append(i.split(';')[0])  
                if str(user_id) not in list_id: # если пользователь ещё не записан (отсутствует в файле)
                    vk.method("messages.send", {"peer_id": id, "message": 'Ты ещё не записан(-а)', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard.get_keyboard()})  
                        
                elif str(user_id) in list_id:
                  for j in data_list:
                    if str(user_id) == j.split(';')[0]:   # если уже записывался 
                       data_list.remove(j)
                       vshp = j[j.rfind(';')+2:]
                       name = ' '.join(j.split(';')[1:4])
                       message ='Отмена записи: ' + name
                       vk.method("messages.send", {"peer_id": id, "message": "Твоя запись успешно отменена", "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})
                       vk.method("messages.send", {"peer_id": 572556244, "message": message, "random_id": random.randint(1, 2147483647)})   # переселыка в лс о отмене человека  
                       vk.method("messages.send", {"peer_id": 572556244, "message": str(data_list), "random_id": random.randint(1, 2147483647)})   # переселыка списка записавшихся в лс о отмене человека  
                       break
                  writing_file(data_list)
                

          # 2 уровень (запись на воркшоп)
            elif object["payload"] == '{"type":"v2_v1"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Выбери воркшоп', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard_seria1.get_keyboard()})  
            elif object["payload"] == '{"type":"v2_v2"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Выбери воркшоп', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard_seria2.get_keyboard()})  
            elif object["payload"] == '{"type":"v2_v3"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Выбери действие', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard.get_keyboard()})
        
        # 2 уровень (инфа)
            elif object["payload"] == '{"type":"v3_v1"}':   
                vk.method("messages.send", {"peer_id": id, "message": '*возвращается пдф файл*', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
            elif object["payload"] == '{"type":"v3_v2"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Место: карта: ', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
            elif object["payload"] == '{"type":"v3_v3"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Онлайн-трекер', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
            elif object["payload"] == '{"type":"v3_v4"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Выбери действие', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
        
       # 3 уровень (серия 1)
            elif object["payload"] == '{"type":"v4_v1"}':                 
                data_list.append(str(user_id) + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['first_name'] + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['last_name'] + '; ' + 'Воркшоп 1')
                writing_file(data_list)
                for i in range(len(data_list)):
                    if data_list[i].split(';')[0] == str(user_id):
                        name = data_list[i].split(';')[1:4] # имя, фамилия и воркшоп, на который записался пользователь
                        message = 'Новая запись:' + ' '.join(name)
                vk.method("messages.send", {"peer_id": id, "message": 'Воркшоп пройдёт в аудитории 11 ', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
                vk.method("messages.send", {"peer_id": 572556244, "message": message, "random_id": random.randint(2, 2147483647)})  # пересылка себе в лс о новой записи 
                vk.method("messages.send", {"peer_id": 572556244, "message": str(data_list), "random_id": random.randint(1, 2147483647)}) 
            elif object["payload"] == '{"type":"v4_v2"}':             
                data_list.append(str(user_id) + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['first_name'] + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['last_name'] + '; ' + 'Воркшоп 2')
                writing_file(data_list)
                for i in range(len(data_list)):
                    if data_list[i].split(';')[0] == str(user_id):
                        name = data_list[i].split(';')[1:4] # имя, фамилия и воркшоп, на который записался пользователь
                        message = 'Новая запись:' + ' '.join(name)
                    
                vk.method("messages.send", {"peer_id": id, "message": 'Воркшоп пройдёт в аудитории 12 ', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
                vk.method("messages.send", {"peer_id": 572556244, "message": message, "random_id": random.randint(2, 2147483647)})  # пересылка себе в лс о новой записи 
                vk.method("messages.send", {"peer_id": 572556244, "message": str(data_list), "random_id": random.randint(1, 2147483647)}) 
            elif object["payload"] == '{"type":"v4_v3"}':   
                data_list.append(str(user_id) + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['first_name'] + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['last_name'] + '; ' + 'Воркшоп 3')
                writing_file(data_list)
                for i in range(len(data_list)):
                    if data_list[i].split(';')[0] == str(user_id):
                       name = data_list[i].split(';')[1:4] # имя, фамилия и воркшоп, на который записался пользователь
                       message = 'Новая запись:' + ' '.join(name)
                vk.method("messages.send", {"peer_id": id, "message": 'Воркшоп пройдёт в аудитории 13 ', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
                vk.method("messages.send", {"peer_id": 572556244, "message": message, "random_id": random.randint(2, 2147483647)})  # пересылка себе в лс о новой записи   
                vk.method("messages.send", {"peer_id": 572556244, "message": str(data_list), "random_id": random.randint(1, 2147483647)}) 
            elif object["payload"] == '{"type":"v4_v4"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Выбери серию, на которую хочешь записаться', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard_vorkshop.get_keyboard()})
    
       # 3 уровень (серия 2)
            elif object["payload"] == '{"type":"v5_v1"}':             
                data_list.append(str(user_id) + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['first_name'] + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['last_name'] + '; ' + 'Воркшоп 4')
                writing_file(data_list)
                for i in range(len(data_list)):
                    if data_list[i].split(';')[0] == str(user_id):
                        name = data_list[i].split(';')[1:4] # имя, фамилия и воркшоп, на который записался пользователь
                        message = 'Новая запись:' + ' '.join(name)
                vk.method("messages.send", {"peer_id": id, "message": 'Воркшоп пройдёт в аудитории 21 ', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
                vk.method("messages.send", {"peer_id": 572556244, "message": message, "random_id": random.randint(2, 2147483647)})  # пересылка себе в лс о новой записи 
                vk.method("messages.send", {"peer_id": 572556244, "message": str(data_list), "random_id": random.randint(1, 2147483647)}) 
            elif object["payload"] == '{"type":"v5_v3"}':                   
                data_list.append(str(user_id) + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['first_name'] + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['last_name'] + '; ' + 'Воркшоп 6')
                writing_file(data_list) 
                for i in range(len(data_list)):
                    if data_list[i].split(';')[0] == str(user_id):
                        name = data_list[i].split(';')[1:4] # имя, фамилия и воркшоп, на который записался пользователь
                        message = 'Новая запись:' + ' '.join(name)
                vk.method("messages.send", {"peer_id": id, "message": 'Воркшоп пройдёт в аудитории 23 ', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
                vk.method("messages.send", {"peer_id": 572556244, "message": message, "random_id": random.randint(2, 2147483647)})  # пересылка себе в лс о новой записи   
                vk.method("messages.send", {"peer_id": 572556244, "message": str(data_list), "random_id": random.randint(1, 2147483647)})           
            
            elif object["payload"] == '{"type":"v5_v2"}':                 
                data_list.append(str(user_id) + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['first_name'] + '; ' + vk.method("users.get", {"user_ids": user_id})[0]['last_name'] + '; ' + 'Воркшоп 5')
                writing_file(data_list)
                for i in data_list:
                    if i.split(';')[0] == str(user_id):
                        name = i.split(';')[1:4] # имя, фамилия и воркшоп, на который записался пользователь
                        message = 'Новая запись:' + ' '.join(name)
                vk.method("messages.send", {"peer_id": id, "message": 'Воркшоп пройдёт в аудитории 22 ', "random_id": random.randint(2, 2147483647), "keyboard": keyboard.get_keyboard()})  
                vk.method("messages.send", {"peer_id": 572556244, "message": message, "random_id": random.randint(2, 2147483647)})  # пересылка себе в лс о новой записи 
                vk.method("messages.send", {"peer_id": 572556244, "message": str(data_list), "random_id": random.randint(1, 2147483647)})  
            elif object["payload"] == '{"type":"v5_v4"}':   
                vk.method("messages.send", {"peer_id": id, "message": 'Выбери серию, на которую хочешь записаться', "random_id": random.randint(2, 2147483647),  "keyboard": keyboard_vorkshop.get_keyboard()})
                                
    except:
            vk.method("messages.send", {"peer_id": 572556244, "message": traceback.format_exc(), "random_id": random.randint(1, 2147483647)})

    return "ok"

# if __name__ == "__main__":
#    application.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
